<a href="https://colab.research.google.com/github/yoouza/bigcontest/blob/master/data_preprocessing/News_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import pandas as pd
import json, os, re

from tqdm import tqdm
from datetime import datetime
from nltk.tokenize import sent_tokenize

# **데이터 불러오기**

In [3]:
path = '/content/drive/My Drive/머신러닝/팀 프로젝트/빅콘테스트/00. Data/07. 뉴스'
files = os.listdir(path)

news = []
for file in tqdm(files):
    with open(path+'/'+file, 'rb') as f:
        for item in json.load(f):
            news_content = {'date':0, 'title':0, 'text':0}
            news_content['date'] = item['date']
            news_content['title'] = item['title']
            news_content['text'] = item['text']
            news.append(news_content)

100%|██████████| 5/5 [00:57<00:00, 11.60s/it]


In [4]:
news_df = pd.DataFrame(news)
news_df.head(), len(news_df)

(         date  ...                                               text
 0  2019-02-01  ...  EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중  (브뤼셀·서울=연합뉴스)...
 1  2019-02-01  ...  [세종=이데일리 김형욱 기자] ◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승...
 2  2019-02-01  ...  (뉴욕=연합뉴스) 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증...
 3  2019-02-01  ...  방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것”      ...
 4  2019-02-01  ...  [세종=이데일리 김형욱 기자] ◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상...
 
 [5 rows x 3 columns], 605268)

# **데이터 전처리**

In [5]:
news_df = news_df.drop_duplicates(['text']) # 본문 중복 제거
news_df = news_df.reset_index()
news_df = news_df.drop(['index'], axis=1)
len(news_df)

372384

In [6]:
news_df = news_df.fillna("")
news_df.head()

,date,title,text
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 (브뤼셀·서울=연합뉴스)...
1,2019-02-01,[인사]산업통상자원부,[세종=이데일리 김형욱 기자] ◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승...
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,(뉴욕=연합뉴스) 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증...
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” ...
4,2019-02-01,[인사]한국로봇산업진흥원,[세종=이데일리 김형욱 기자] ◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상...


In [7]:
news_df['text'] = news_df.text.apply(lambda x : x.replace('\\t', '').replace('\n', '').replace('  ', ' ').replace('   ', ' '))
news_df['text'] = news_df.text.apply(lambda x : re.sub('[ A-Za-z0-9]*\@[a-z]*\.[a-z]*.kr', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('[ A-Za-z0-9]*\@[a-z]*\.[a-z]*', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('이데일리 채널 구독하면 [방탄소년단 실물영접 기회가]', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('이데일리 채널 구독하면 \[방탄소년단 실물영접 기회가', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('네이버에서 한국경제 채널 구독하기', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('ⓒ 한국경제 &amp; hankyung.com , 무단전재 및 재배포 금지', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('저작권자 ⓒ 뉴스1코리아, 무단전재 및 재배포 금지', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('&lt;자세히 보기&gt;', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\[[가-히a-zA-Z=·, @….0-9/:]*\]', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\([가-히a-zA-Z=·, @….0-9/:]*\)', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\【[가-히a-zA-Z=·, @….0-9/:]*\】', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\<[가-히a-zA-Z=·, @….]*\>', '', x))
#news_df['text'] = news_df.loc[news_df['text'].apply(lambda x:len(x)) >= 10, "text"] # 문서 길이 10 이상인 뉴스
news_df.head()

,date,title,text
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영 기...
1,2019-02-01,[인사]산업통상자원부,◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승진 △장관정책보좌관 김완기 ◇...
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증가세를 보였다. ...
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” 김현종 ...
4,2019-02-01,[인사]한국로봇산업진흥원,◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상미 △시설안전보안팀장 김덕구 ...


In [8]:
news_df['text'] = news_df.text.apply(lambda x : sent_tokenize(x))
news_df.head()

,date,title,text
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),[EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영 ...
1,2019-02-01,[인사]산업통상자원부,[ ◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승진 △장관정책보좌관 김완기 ...
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,[ 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증가세를 보였다....
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",[방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” 김현종...
4,2019-02-01,[인사]한국로봇산업진흥원,[ ◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상미 △시설안전보안팀장 김덕구...


# **데이터 라벨링**

In [9]:
news_df['new_date'] = 0
news_df['new_date'] = news_df.date.apply(lambda x : str(datetime.strptime(x, "%Y-%m-%d"))[:10])
news_df.head()

,date,title,text,new_date
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),[EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영 ...,2019-02-01
1,2019-02-01,[인사]산업통상자원부,[ ◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승진 △장관정책보좌관 김완기 ...,2019-02-01
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,[ 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증가세를 보였다....,2019-02-01
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",[방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” 김현종...,2019-02-01
4,2019-02-01,[인사]한국로봇산업진흥원,[ ◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상미 △시설안전보안팀장 김덕구...,2019-02-01


In [10]:
acco_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_10_2019_ud.csv'
acco_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_10_2020_ud.csv'
lesu_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_20_2019_ud.csv'
lesu_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_20_2020_ud.csv'
furn_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_30_2019_ud.csv'
furn_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_30_2020_ud.csv'
kitc_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_32_2019_ud.csv'
kitc_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_32_2020_ud.csv'
home_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_35_2019_ud.csv'
home_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_35_2020_ud.csv'
tele_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_52_2019_ud.csv'
tele_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_52_2020_ud.csv'
stor_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_40_2019_ud.csv'
stor_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_40_2020_ud.csv'
clot_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_42_2019_ud.csv'
clot_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_42_2020_ud.csv'
jewl_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_44_2019_ud.csv'
jewl_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_44_2020_ud.csv'
rest_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_80_2019_ud.csv'
rest_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_80_2020_ud.csv'
food_2019_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_81_2019_ud.csv'
food_2020_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/카테고리별 3일 윈도우 업다운/code_81_2020_ud.csv'

In [11]:
acco_df_2019 = pd.read_csv(acco_2019_path, index_col=0)
acco_df_2020 = pd.read_csv(acco_2020_path, index_col=0)
lesu_df_2019 = pd.read_csv(lesu_2019_path, index_col=0)
lesu_df_2020 = pd.read_csv(lesu_2020_path, index_col=0)
furn_df_2019 = pd.read_csv(furn_2019_path, index_col=0)
furn_df_2020 = pd.read_csv(furn_2020_path, index_col=0)
kitc_df_2019 = pd.read_csv(kitc_2019_path, index_col=0)
kitc_df_2020 = pd.read_csv(kitc_2020_path, index_col=0)
home_df_2019 = pd.read_csv(home_2019_path, index_col=0)
home_df_2020 = pd.read_csv(home_2020_path, index_col=0)
tele_df_2019 = pd.read_csv(tele_2019_path, index_col=0)
tele_df_2020 = pd.read_csv(tele_2020_path, index_col=0)
stor_df_2019 = pd.read_csv(stor_2019_path, index_col=0)
stor_df_2020 = pd.read_csv(stor_2020_path, index_col=0)
clot_df_2019 = pd.read_csv(clot_2019_path, index_col=0)
clot_df_2020 = pd.read_csv(clot_2020_path, index_col=0)
jewl_df_2019 = pd.read_csv(jewl_2019_path, index_col=0)
jewl_df_2020 = pd.read_csv(jewl_2020_path, index_col=0)
rest_df_2019 = pd.read_csv(rest_2019_path, index_col=0)
rest_df_2020 = pd.read_csv(rest_2020_path, index_col=0)
food_df_2019 = pd.read_csv(food_2019_path, index_col=0)
food_df_2020 = pd.read_csv(food_2020_path, index_col=0)

## **숙박**

In [12]:
acco_df = pd.concat([acco_df_2019, acco_df_2020])
acco_df = acco_df.reset_index()
acco_df.head()

,index,date,USE_AMT,3d_later,ud
0,2,2019-02-03,1550921.0,1510619.0,0
1,3,2019-02-04,1701842.0,1335314.0,0
2,4,2019-02-05,1609012.0,1195248.0,0
3,5,2019-02-06,1510619.0,1501568.0,0
4,6,2019-02-07,1335314.0,1700831.0,1


In [13]:
# 날짜 데이트화
acco_df['new_date'] = str(0)
for i in range(len(acco_df)):
    acco_df['new_date'][i] = str(datetime.strptime(acco_df['date'][i], "%Y-%m-%d"))[:10]

acco_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,index,date,USE_AMT,3d_later,ud,new_date
0,2,2019-02-03,1550921.0,1510619.0,0,2019-02-03
1,3,2019-02-04,1701842.0,1335314.0,0,2019-02-04
2,4,2019-02-05,1609012.0,1195248.0,0,2019-02-05
3,5,2019-02-06,1510619.0,1501568.0,0,2019-02-06
4,6,2019-02-07,1335314.0,1700831.0,1,2019-02-07


In [14]:
diff = acco_df[['new_date', 'ud']]
diff.head()

,new_date,ud
0,2019-02-03,0
1,2019-02-04,0
2,2019-02-05,0
3,2019-02-06,0
4,2019-02-07,1


In [15]:
news_wt_diff = pd.merge(diff, news_df, how = 'left', on='new_date')
news_wt_diff = news_wt_diff.sort_values(by = ['date'], axis=0, ascending=True)
news_wt_diff = news_wt_diff.reset_index()
news_wt_diff = news_wt_diff.drop(['index'], axis=1)
news_wt_diff.head()

,new_date,ud,date,title,text
0,2019-02-03,0,2019-02-03,"'카풀 정국', 제자리걸음 멈출 수 있을까",['택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규...
1,2019-02-03,0,2019-02-03,"토트넘, 뉴캐슬에 1-0 승리…손흥민 '10호골'",[ 손흥민이 2일 영국 런던 웸블리스타디움에서 열린 잉글랜드 프로축구 프리미어리...
2,2019-02-03,0,2019-02-03,비 내리는 텅 빈 도심,[ 김인철 기자 = 설 연휴인 3일 오전 서울 종로구 광화문광장 인근에 비가 내리고...
3,2019-02-03,0,2019-02-03,"""외국인母가 낳은 자녀도 사랑이법 적용""…미혼부 출생신고 가능",[생모 인적사항 확인할 수 없다면 생부가 출생신고 가능 출생신고 박초롱 기자 =...
4,2019-02-03,0,2019-02-03,과속하다 행인 치어 의식불명 빠뜨린 오토바이 배달원 법정구속,[법원 피해자 가족 엄벌 탄원…상응 처벌 필요해 금고 10개월 전창해 기자 = 오...


In [16]:
data = news_wt_diff[['new_date', 'text', 'ud']]
data = data.dropna()
data = data.reset_index()
data.head()

,index,new_date,text,ud
0,0,2019-02-03,['택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규...,0
1,1,2019-02-03,[ 손흥민이 2일 영국 런던 웸블리스타디움에서 열린 잉글랜드 프로축구 프리미어리...,0
2,2,2019-02-03,[ 김인철 기자 = 설 연휴인 3일 오전 서울 종로구 광화문광장 인근에 비가 내리고...,0
3,3,2019-02-03,[생모 인적사항 확인할 수 없다면 생부가 출생신고 가능 출생신고 박초롱 기자 =...,0
4,4,2019-02-03,[법원 피해자 가족 엄벌 탄원…상응 처벌 필요해 금고 10개월 전창해 기자 = 오...,0


In [17]:
new_data = []

for i in tqdm(range(len(data))):
    for j in range(len(data['text'][i])):
        tmp_data = {'date' : 0, 'content' : 0, 'label' : 0}
        tmp_data['date'] = data['new_date'][i]
        tmp_data['content'] = data['text'][i][j]
        tmp_data['label'] = data['ud'][i]
        new_data.append(tmp_data)

new_data[:5]

100%|██████████| 238255/238255 [02:35<00:00, 1533.46it/s]


[{'content': "'택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규제 해소 등 개선방안과 카풀 공존 방법 논의 서둘러야  카카오의 카풀 자회사 카카오모빌리티가 카풀 시범 서비스를 잠정 중단한 18일 서울 여의도 국회 앞 카풀 반대 분신 택시기사 분향소가 고요한 모습을 보이고 있다.",
  'date': '2019-02-03',
  'label': 0},
 {'content': '/문호남 기자 munonam@  택시업계와 정부, 업계가 지난한 과정 끝에 처음으로 협상 테이블에 앉았다.',
  'date': '2019-02-03',
  'label': 0},
 {'content': "이들이 이끌어낸 첫 합의는 '자가용이 아닌 택시로 공유경제를 이루겠다'였다.",
  'date': '2019-02-03',
  'label': 0},
 {'content': '금지된 택시 합승을 플랫폼 사업자가 중개하는 것이 아니냐는 비판이 빗발쳤다.',
  'date': '2019-02-03',
  'label': 0},
 {'content': '결국 카풀 논의는 여전히 제자리걸음에 머물러있다.', 'date': '2019-02-03', 'label': 0}]

In [18]:
data_accom = pd.DataFrame(new_data)
data_accom.head()

,date,content,label
0,2019-02-03,'택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규제...,0
1,2019-02-03,"/문호남 기자 munonam@ 택시업계와 정부, 업계가 지난한 과정 끝에 처음으로...",0
2,2019-02-03,이들이 이끌어낸 첫 합의는 '자가용이 아닌 택시로 공유경제를 이루겠다'였다.,0
3,2019-02-03,금지된 택시 합승을 플랫폼 사업자가 중개하는 것이 아니냐는 비판이 빗발쳤다.,0
4,2019-02-03,결국 카풀 논의는 여전히 제자리걸음에 머물러있다.,0


## **레저용품**

In [19]:
def attach_label(df1, df2):
    total_df = pd.concat([df1, df2])
    df1 = df1.reset_index()

    # 날짜 데이트화
    df1['new_date'] = str(0)
    for i in range(len(df1)):
        df1['new_date'][i] = str(datetime.strptime(df1['date'][i], "%Y-%m-%d"))[:10]
    # 라벨만 달린 날짜
    diff = df1[['new_date', 'ud']]
    # 레프트 조인
    news_wt_diff = pd.merge(diff, news_df, how = 'left', on='new_date')
    news_wt_diff = news_wt_diff.sort_values(by = ['date'], axis=0, ascending=True)
    news_wt_diff = news_wt_diff.reset_index()
    news_wt_diff = news_wt_diff.drop(['index'], axis=1)
    # 필요한 데이터만 가져오기
    data = news_wt_diff[['new_date', 'text', 'ud']]
    data = data.dropna()
    data = data.reset_index()
    # 뉴스 한 줄씩 라벨링
    new_data = []
    for i in tqdm(range(len(data))):
        for j in range(len(data['text'][i])):
            tmp_data = {'date' : 0, 'content' : 0, 'label' : 0}
            tmp_data['date'] = data['new_date'][i]
            tmp_data['content'] = data['text'][i][j]
            tmp_data['label'] = data['ud'][i]
            new_data.append(tmp_data)
    
    return pd.DataFrame(new_data)

In [21]:
data_lesu = attach_label(lesu_df_2019, lesu_df_2020)
data_lesu.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 112497/112497 [01:09<00:00, 1622.40it/s]


,date,content,label
0,2019-02-03,'택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규제...,0
1,2019-02-03,"/문호남 기자 munonam@ 택시업계와 정부, 업계가 지난한 과정 끝에 처음으로...",0
2,2019-02-03,이들이 이끌어낸 첫 합의는 '자가용이 아닌 택시로 공유경제를 이루겠다'였다.,0
3,2019-02-03,금지된 택시 합승을 플랫폼 사업자가 중개하는 것이 아니냐는 비판이 빗발쳤다.,0
4,2019-02-03,결국 카풀 논의는 여전히 제자리걸음에 머물러있다.,0


## **가구, 주방용품**

In [41]:
fk_2019 = pd.merge(furn_df_2019, kitc_df_2019, on='date')
fk_2019.head()

,date,USE_AMT_x,3d_later_x,ud_x,USE_AMT_y,3d_later_y,ud_y
0,2019-02-03,94818.0,38208.0,0,162525.0,144336.0,0
1,2019-02-04,84866.0,62268.0,0,79507.0,237680.0,1
2,2019-02-06,38208.0,174894.0,1,34915.0,278247.0,1
3,2019-02-07,62268.0,246257.0,1,144336.0,173372.0,1
4,2019-02-08,109021.0,284114.0,1,237680.0,147697.0,0


In [42]:
fk_2019['total_amt'] = 0
fk_2019['total_3d_later'] = 0
for i in range(len(fk_2019)):
    fk_2019['total_amt'][i] = fk_2019['USE_AMT_x'][i] + fk_2019['USE_AMT_y'][i]
    fk_2019['total_3d_later'][i] = fk_2019['3d_later_x'][i] + fk_2019['3d_later_y'][i]
fk_2019.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,date,USE_AMT_x,3d_later_x,ud_x,USE_AMT_y,3d_later_y,ud_y,total_amt,total_3d_later
0,2019-02-03,94818.0,38208.0,0,162525.0,144336.0,0,257343,182544
1,2019-02-04,84866.0,62268.0,0,79507.0,237680.0,1,164373,299948
2,2019-02-06,38208.0,174894.0,1,34915.0,278247.0,1,73123,453141
3,2019-02-07,62268.0,246257.0,1,144336.0,173372.0,1,206604,419629
4,2019-02-08,109021.0,284114.0,1,237680.0,147697.0,0,346701,431811


In [43]:
# 전체 매출량 차이 라벨링(전날 대비 상승, 하강, 변화없음)
fk_2019['ud'] = 0
fk_2019['diff'] = 0
for i in range(len(fk_2019)):
    fk_2019['diff'][i] = fk_2019['total_3d_later'][i] - fk_2019['total_amt'][i]
    if fk_2019['diff'][i] < 0:
        fk_2019['ud'][i] = 0
    elif fk_2019['diff'][i] > 0:
        fk_2019['ud'][i] = 1
fk_2019.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,date,USE_AMT_x,3d_later_x,ud_x,USE_AMT_y,3d_later_y,ud_y,total_amt,total_3d_later,ud,diff
0,2019-02-03,94818.0,38208.0,0,162525.0,144336.0,0,257343,182544,0,-74799
1,2019-02-04,84866.0,62268.0,0,79507.0,237680.0,1,164373,299948,1,135575
2,2019-02-06,38208.0,174894.0,1,34915.0,278247.0,1,73123,453141,1,380018
3,2019-02-07,62268.0,246257.0,1,144336.0,173372.0,1,206604,419629,1,213025
4,2019-02-08,109021.0,284114.0,1,237680.0,147697.0,0,346701,431811,1,85110


In [44]:
fk_2020 = pd.merge(furn_df_2020, kitc_df_2020, on='date')

fk_2020['total_amt'] = 0
fk_2020['total_3d_later'] = 0
for i in range(len(fk_2020)):
    fk_2020['total_amt'][i] = fk_2020['USE_AMT_x'][i] + fk_2020['USE_AMT_y'][i]
    fk_2020['total_3d_later'][i] = fk_2020['3d_later_x'][i] + fk_2020['3d_later_y'][i]

# 전체 매출량 차이 라벨링(전날 대비 상승, 하강)
fk_2020['ud'] = 0
fk_2020['diff'] = 0
for i in range(len(fk_2020)):
    fk_2020['diff'][i] = fk_2020['total_3d_later'][i] - fk_2020['total_amt'][i]
    if fk_2020['diff'][i] < 0:
        fk_2020['ud'][i] = 0
    elif fk_2020['diff'][i] > 0:
        fk_2020['ud'][i] = 1
fk_2020.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopy

,date,USE_AMT_x,3d_later_x,ud_x,USE_AMT_y,3d_later_y,ud_y,total_amt,total_3d_later,ud,diff
0,2020-02-03,273914.0,123051.0,0,218778.0,202215.0,0,492692,325266,0,-167426
1,2020-02-04,190349.0,130358.0,0,255163.0,164455.0,0,445512,294813,0,-150699
2,2020-02-05,167328.0,223115.0,1,301473.0,137782.0,0,468801,360897,0,-107904
3,2020-02-06,123051.0,255290.0,1,202215.0,113353.0,0,325266,368643,1,43377
4,2020-02-07,130358.0,258769.0,1,164455.0,149450.0,0,294813,408219,1,113406


In [45]:
data_fk = attach_label(fk_2019, fk_2020)
data_fk.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 112190/112190 [01:15<00:00, 1491.57it/s]


,date,content,label
0,2019-02-03,'택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규제...,0
1,2019-02-03,"/문호남 기자 munonam@ 택시업계와 정부, 업계가 지난한 과정 끝에 처음으로...",0
2,2019-02-03,이들이 이끌어낸 첫 합의는 '자가용이 아닌 택시로 공유경제를 이루겠다'였다.,0
3,2019-02-03,금지된 택시 합승을 플랫폼 사업자가 중개하는 것이 아니냐는 비판이 빗발쳤다.,0
4,2019-02-03,결국 카풀 논의는 여전히 제자리걸음에 머물러있다.,0


## **가전, 사무통신**

In [47]:
ht_2019 = pd.merge(home_df_2019, tele_df_2019, on='date')

ht_2019['total_amt'] = 0
ht_2019['total_3d_later'] = 0
for i in range(len(ht_2019)):
    ht_2019['total_amt'][i] = ht_2019['USE_AMT_x'][i] + ht_2019['USE_AMT_y'][i]
    ht_2019['total_3d_later'][i] = ht_2019['3d_later_x'][i] + ht_2019['3d_later_y'][i]

# 전체 매출량 차이 라벨링(전날 대비 상승, 하강)
ht_2019['ud'] = 0
ht_2019['diff'] = 0
for i in range(len(ht_2019)):
    ht_2019['diff'][i] = ht_2019['total_3d_later'][i] - ht_2019['total_amt'][i]
    if ht_2019['diff'][i] < 0:
        ht_2019['ud'][i] = 0
    elif ht_2019['diff'][i] > 0:
        ht_2019['ud'][i] = 1
ht_2019.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopy

,date,USE_AMT_x,3d_later_x,ud_x,USE_AMT_y,3d_later_y,ud_y,total_amt,total_3d_later,ud,diff
0,2019-02-03,820559.0,341036.0,0,1956213.0,1412692.0,0,2776772,1753728,0,-1023044
1,2019-02-04,679987.0,560192.0,0,1766603.0,1387728.0,0,2446590,1947920,0,-498670
2,2019-02-05,380845.0,768725.0,1,1528675.0,1532507.0,1,1909520,2301232,1,391712
3,2019-02-06,341036.0,923215.0,1,1412692.0,1658821.0,1,1753728,2582036,1,828308
4,2019-02-07,560192.0,753433.0,1,1387728.0,1581829.0,1,1947920,2335262,1,387342


In [48]:
ht_2020 = pd.merge(home_df_2020, tele_df_2020, on='date')

ht_2020['total_amt'] = 0
ht_2020['total_3d_later'] = 0
for i in range(len(ht_2020)):
    ht_2020['total_amt'][i] = ht_2020['USE_AMT_x'][i] + ht_2020['USE_AMT_y'][i]
    ht_2020['total_3d_later'][i] = ht_2020['3d_later_x'][i] + ht_2020['3d_later_y'][i]

# 전체 매출량 차이 라벨링(전날 대비 상승, 하강)
ht_2020['ud'] = 0
ht_2020['diff'] = 0
for i in range(len(ht_2020)):
    ht_2020['diff'][i] = ht_2020['total_3d_later'][i] - ht_2020['total_amt'][i]
    if ht_2020['diff'][i] < 0:
        ht_2020['ud'][i] = 0
    elif ht_2020['diff'][i] > 0:
        ht_2020['ud'][i] = 1
ht_2020.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopy

,date,USE_AMT_x,3d_later_x,ud_x,USE_AMT_y,3d_later_y,ud_y,total_amt,total_3d_later,ud,diff
0,2020-02-03,748204.0,331946.0,0,1620420.0,1806289.0,1,2368624,2138235,0,-230389
1,2020-02-04,558540.0,386201.0,0,1630507.0,1923854.0,1,2189047,2310055,1,121008
2,2020-02-05,445256.0,540255.0,1,1749998.0,1944201.0,1,2195254,2484456,1,289202
3,2020-02-06,331946.0,639296.0,1,1806289.0,1817157.0,1,2138235,2456453,1,318218
4,2020-02-07,386201.0,672498.0,1,1923854.0,1671713.0,0,2310055,2344211,1,34156


In [49]:
data_ht = attach_label(ht_2019, ht_2020)
data_ht.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 112497/112497 [01:12<00:00, 1551.51it/s]


,date,content,label
0,2019-02-03,'택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규제...,0
1,2019-02-03,"/문호남 기자 munonam@ 택시업계와 정부, 업계가 지난한 과정 끝에 처음으로...",0
2,2019-02-03,이들이 이끌어낸 첫 합의는 '자가용이 아닌 택시로 공유경제를 이루겠다'였다.,0
3,2019-02-03,금지된 택시 합승을 플랫폼 사업자가 중개하는 것이 아니냐는 비판이 빗발쳤다.,0
4,2019-02-03,결국 카풀 논의는 여전히 제자리걸음에 머물러있다.,0


## **유통**

In [50]:
data_stor = attach_label(stor_df_2019, stor_df_2020)
data_stor.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 112497/112497 [01:12<00:00, 1555.59it/s]


,date,content,label
0,2019-02-03,'택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규제...,0
1,2019-02-03,"/문호남 기자 munonam@ 택시업계와 정부, 업계가 지난한 과정 끝에 처음으로...",0
2,2019-02-03,이들이 이끌어낸 첫 합의는 '자가용이 아닌 택시로 공유경제를 이루겠다'였다.,0
3,2019-02-03,금지된 택시 합승을 플랫폼 사업자가 중개하는 것이 아니냐는 비판이 빗발쳤다.,0
4,2019-02-03,결국 카풀 논의는 여전히 제자리걸음에 머물러있다.,0


## **의복**

In [51]:
data_clot = attach_label(clot_df_2019, clot_df_2020)
data_clot.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 112497/112497 [01:11<00:00, 1565.89it/s]


,date,content,label
0,2019-02-03,'택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규제...,0
1,2019-02-03,"/문호남 기자 munonam@ 택시업계와 정부, 업계가 지난한 과정 끝에 처음으로...",0
2,2019-02-03,이들이 이끌어낸 첫 합의는 '자가용이 아닌 택시로 공유경제를 이루겠다'였다.,0
3,2019-02-03,금지된 택시 합승을 플랫폼 사업자가 중개하는 것이 아니냐는 비판이 빗발쳤다.,0
4,2019-02-03,결국 카풀 논의는 여전히 제자리걸음에 머물러있다.,0


## **신변잡화**

In [52]:
data_jewl = attach_label(jewl_df_2019, jewl_df_2020)
data_jewl.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 112497/112497 [01:13<00:00, 1536.97it/s]


,date,content,label
0,2019-02-03,'택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규제...,0
1,2019-02-03,"/문호남 기자 munonam@ 택시업계와 정부, 업계가 지난한 과정 끝에 처음으로...",0
2,2019-02-03,이들이 이끌어낸 첫 합의는 '자가용이 아닌 택시로 공유경제를 이루겠다'였다.,0
3,2019-02-03,금지된 택시 합승을 플랫폼 사업자가 중개하는 것이 아니냐는 비판이 빗발쳤다.,0
4,2019-02-03,결국 카풀 논의는 여전히 제자리걸음에 머물러있다.,0


## **요식업**

In [54]:
data_rest = attach_label(rest_df_2019, rest_df_2020)
data_rest.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 112497/112497 [01:13<00:00, 1538.13it/s]


,date,content,label
0,2019-02-03,'택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규제...,0
1,2019-02-03,"/문호남 기자 munonam@ 택시업계와 정부, 업계가 지난한 과정 끝에 처음으로...",0
2,2019-02-03,이들이 이끌어낸 첫 합의는 '자가용이 아닌 택시로 공유경제를 이루겠다'였다.,0
3,2019-02-03,금지된 택시 합승을 플랫폼 사업자가 중개하는 것이 아니냐는 비판이 빗발쳤다.,0
4,2019-02-03,결국 카풀 논의는 여전히 제자리걸음에 머물러있다.,0


## **음식품**

In [53]:
data_food = attach_label(food_df_2019, food_df_2020)
data_food.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 112497/112497 [01:10<00:00, 1605.52it/s]


,date,content,label
0,2019-02-03,'택시 이용 카풀' 합의에 여론은 싸늘 실제 이용자 의견 반영 필요 지적 택시 규제...,0
1,2019-02-03,"/문호남 기자 munonam@ 택시업계와 정부, 업계가 지난한 과정 끝에 처음으로...",0
2,2019-02-03,이들이 이끌어낸 첫 합의는 '자가용이 아닌 택시로 공유경제를 이루겠다'였다.,0
3,2019-02-03,금지된 택시 합승을 플랫폼 사업자가 중개하는 것이 아니냐는 비판이 빗발쳤다.,0
4,2019-02-03,결국 카풀 논의는 여전히 제자리걸음에 머물러있다.,0


# **데이터 저장**

In [56]:
data_accom.to_csv('/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_accomodation.csv')
data_lesu.to_csv('/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_leisure.csv')
data_fk.to_csv('/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_furniture_kitchen.csv')
data_ht.to_csv('/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_home_tele.csv')
data_stor.to_csv('/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_store.csv')
data_clot.to_csv('/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_clothes.csv')
data_jewl.to_csv('/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_jewlry.csv')
data_rest.to_csv('/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_restaurant.csv')
data_food.to_csv('/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_food.csv')